In [2]:
import keras
from keras.utils.np_utils import to_categorical

import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('./DATA/sensorless_data.csv')
data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F40,F41,F42,F43,F44,F45,F46,F47,F48,TARGET
0,-3.014600e-07,8.260300e-06,-0.000012,-0.000002,-1.438600e-06,-0.000021,0.031718,0.031710,0.031721,-0.032963,...,-0.63308,2.9646,8.1198,-1.4961,-1.4961,-1.4961,-1.4996,-1.4996,-1.4996,1
1,2.913200e-06,-5.247700e-06,0.000003,-0.000006,2.778900e-06,-0.000004,0.030804,0.030810,0.030806,-0.033520,...,-0.59314,7.6252,6.1690,-1.4967,-1.4967,-1.4967,-1.5005,-1.5005,-1.5005,1
2,-2.951700e-06,-3.184000e-06,-0.000016,-0.000001,-1.575300e-06,0.000017,0.032877,0.032880,0.032896,-0.029834,...,-0.63252,2.7784,5.3017,-1.4983,-1.4983,-1.4982,-1.4985,-1.4985,-1.4985,1
3,-1.322600e-06,8.820100e-06,-0.000016,-0.000005,-7.282900e-07,0.000004,0.029410,0.029401,0.029417,-0.030156,...,-0.62289,6.5534,6.2606,-1.4963,-1.4963,-1.4963,-1.4975,-1.4975,-1.4976,1
4,-6.836600e-08,5.666300e-07,-0.000026,-0.000006,-7.940600e-07,0.000013,0.030119,0.030119,0.030145,-0.031393,...,-0.63010,4.5155,9.5231,-1.4958,-1.4958,-1.4958,-1.4959,-1.4959,-1.4959,1


### Format input data

In [4]:
x_cols = list(data.columns[:-1])
X_data = data[x_cols].copy()
# Adding 0 for easy reshaping
X_data['F49'] = 0
X_data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F40,F41,F42,F43,F44,F45,F46,F47,F48,F49
0,-3.014600e-07,8.260300e-06,-0.000012,-0.000002,-1.438600e-06,-0.000021,0.031718,0.031710,0.031721,-0.032963,...,-0.63308,2.9646,8.1198,-1.4961,-1.4961,-1.4961,-1.4996,-1.4996,-1.4996,0
1,2.913200e-06,-5.247700e-06,0.000003,-0.000006,2.778900e-06,-0.000004,0.030804,0.030810,0.030806,-0.033520,...,-0.59314,7.6252,6.1690,-1.4967,-1.4967,-1.4967,-1.5005,-1.5005,-1.5005,0
2,-2.951700e-06,-3.184000e-06,-0.000016,-0.000001,-1.575300e-06,0.000017,0.032877,0.032880,0.032896,-0.029834,...,-0.63252,2.7784,5.3017,-1.4983,-1.4983,-1.4982,-1.4985,-1.4985,-1.4985,0
3,-1.322600e-06,8.820100e-06,-0.000016,-0.000005,-7.282900e-07,0.000004,0.029410,0.029401,0.029417,-0.030156,...,-0.62289,6.5534,6.2606,-1.4963,-1.4963,-1.4963,-1.4975,-1.4975,-1.4976,0
4,-6.836600e-08,5.666300e-07,-0.000026,-0.000006,-7.940600e-07,0.000013,0.030119,0.030119,0.030145,-0.031393,...,-0.63010,4.5155,9.5231,-1.4958,-1.4958,-1.4958,-1.4959,-1.4959,-1.4959,0


In [5]:
len(X_data)

58509

In [6]:
X = X_data.to_numpy().reshape((len(X_data),7,7,1))
X.shape

(58509, 7, 7, 1)

In [7]:
X[0].shape

(7, 7, 1)

In [8]:
data['TARGET'] = data['TARGET'] - 1
y = data['TARGET'].to_numpy()

In [9]:
y = to_categorical(y, num_classes=data['TARGET'].nunique())

### Randomizing dataset

In [10]:
idxs = np.arange(len(X))
samples = np.random.choice(idxs,size=10000)

X_rand = X[samples]
y_rand = y[samples]

### CNN Model

In [11]:
model = keras.models.Sequential([
    keras.layers.Conv2D(256, (2,2), strides=1, activation='relu', input_shape=(7,7,1)),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    keras.layers.Conv2D(256, (2,2), strides=1, activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(11, activation='softmax')
])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 6, 6, 256)         1280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 3, 256)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 2, 256)         262400    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 11)                5

In [13]:
y_rand[:25000][0].shape

(11,)

In [14]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_rand , y_rand, steps_per_epoch=100, epochs = 10)

Epoch 1/10
100/100 [==============================] - 1s 12ms/step - loss: 2.4109 - accuracy: 0.1250
Epoch 2/10
100/100 [==============================] - 1s 11ms/step - loss: 1.7962 - accuracy: 0.3457
Epoch 3/10
100/100 [==============================] - 1s 11ms/step - loss: 1.2490 - accuracy: 0.4975
Epoch 4/10
100/100 [==============================] - 1s 10ms/step - loss: 1.1245 - accuracy: 0.5287
Epoch 5/10
100/100 [==============================] - 1s 11ms/step - loss: 0.9581 - accuracy: 0.6174
Epoch 6/10
100/100 [==============================] - 1s 11ms/step - loss: 0.8185 - accuracy: 0.6642
Epoch 7/10
100/100 [==============================] - 1s 14ms/step - loss: 0.7133 - accuracy: 0.6966
Epoch 8/10
100/100 [==============================] - 2s 16ms/step - loss: 0.6550 - accuracy: 0.7286
Epoch 9/10
100/100 [==============================] - 2s 18ms/step - loss: 0.6090 - accuracy: 0.7494
Epoch 10/10
100/100 [==============================] - 1s 12ms/step - loss: 0.5807 - accura

In [15]:
history.history['accuracy'][-1]

0.7598000168800354

### Running Experiment

In [37]:
all_fold_idxs = np.random.choice(idxs, size=len(idxs), replace=False)

nfolds = 3
fold_size = int(len(all_fold_idxs)/3)
folds = [all_fold_idxs[i:i+fold_size] for i in range(0, len(all_fold_idxs), fold_size)]

In [38]:
len(all_fold_idxs)/3

19503.0

In [39]:
len(set(all_fold_idxs))

58509

In [40]:
import pickle

data_dct = {}
data_dct['X'] = X
data_dct['y'] = y
data_dct['folds'] = folds

with open('data_dct.pkl','wb') as f:
    pickle.dump(data_dct,f)

In [41]:
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
for fold_idx in range(nfolds):
    print("Running on fold " + str(fold_idx+1))
    X_test = X[folds[fold_idx]]
    y_test = y[folds[fold_idx]]
    
    train_fold_idxs = [i for i in range(nfolds) if i != fold_idx]
    train_idxs = np.concatenate([list(folds[idx]) for idx in train_fold_idxs])
    X_train = X[train_idxs]
    y_train = y[train_idxs]
    
    model = keras.models.Sequential([
        keras.layers.Conv2D(256, (2,2), strides=1, activation='relu', input_shape=(7,7,1)),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
        keras.layers.Conv2D(256, (2,2), strides=1, activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.Dense(11, activation='softmax')
    ])
    
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
    history = model.fit(X_train , y_train, steps_per_epoch=100, epochs = 10)
    
    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['categorical_accuracy'][-1]

    test_preds = model.predict(X_test)

    loss_calc = keras.metrics.CategoricalCrossentropy()
    loss_calc.reset_states()
    loss_calc.update_state(y_test, test_preds)
    test_loss = loss_calc.result().numpy()

    acc_calc = keras.metrics.CategoricalAccuracy()
    acc_calc.reset_states()
    acc_calc.update_state(y_test, test_preds)
    test_accuracy = acc_calc.result().numpy()

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    # save model
    model.save('CNN_models/model_fold_' + str(fold_idx+1))


Running on fold 1
Epoch 1/10
100/100 [==============================] - 11s 112ms/step - loss: 2.2751 - categorical_accuracy: 0.1835
Epoch 2/10
100/100 [==============================] - 10s 105ms/step - loss: 1.3020 - categorical_accuracy: 0.4878
Epoch 3/10
100/100 [==============================] - 11s 106ms/step - loss: 0.9533 - categorical_accuracy: 0.6257
Epoch 4/10
100/100 [==============================] - 11s 105ms/step - loss: 0.7252 - categorical_accuracy: 0.7017
Epoch 5/10
100/100 [==============================] - 11s 106ms/step - loss: 0.6159 - categorical_accuracy: 0.7528
Epoch 6/10
100/100 [==============================] - 10s 105ms/step - loss: 0.5392 - categorical_accuracy: 0.7774
Epoch 7/10
100/100 [==============================] - 11s 106ms/step - loss: 0.4724 - categorical_accuracy: 0.8023
Epoch 8/10
100/100 [==============================] - 5s 52ms/step - loss: 0.4657 - categorical_accuracy: 0.8018
Epoch 9/10
100/100 [==============================] - 5s 47ms/st

In [42]:
test_accuracies

[0.8326924, 0.81792545, 0.8150541]

In [43]:
np.mean(test_accuracies)

0.82189065